# AutoML Training - Late Delivery Prediction Model

**Goal:** Train ML models to predict late deliveries using AutoML.

This notebook will:
1. Load closed delivery data from the semantic model
2. Train **regression model** to predict AGE_REQ_DATE (days late/early)
3. Create **classification model** for late vs on-time prediction
4. Generate **lateness buckets** (0-2, 3-5, 6-9, 10+ days)
5. Register best models to MLflow

**Use Case:** Enable operations team to:
- Identify deliveries at high risk of shipping late
- Prioritize corrective actions for strategic accounts
- Proactively communicate with business teams about potential delays

### ⭐ 1. Imports

- pandas: dataframe manipulation
- mlflow: experiment tracking and model registry
- AutoML (FLAML): automatic model selection / tuning
- sklearn: train/test split + evaluation metrics
- sempy.fabric: read tables from Power BI semantic model

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Machine Learning
from flaml import AutoML
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_absolute_error, r2_score, mean_squared_error, mean_absolute_percentage_error,
    accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
)
from sklearn.preprocessing import LabelEncoder

# Experiment tracking
import mlflow
from mlflow.tracking import MlflowClient

# Semantic Link - Connect to Power BI
import sempy.fabric as fabric

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ All libraries imported")

### ⭐ 2. Configuration

**IMPORTANT:** Update these settings for your environment.

In [ ]:
# Semantic model name
DATASET = "DLV Aging Columns & Measures"

# Target variable: AGE_REQ_DATE (days late/early vs Customer Requested Delivery Date)
# Positive = Late, Negative = Early, 0 = On-time
TARGET_COLUMN = "AGE_REQ_DATE"

# Model names for MLflow registry
MODEL_NAME_REGRESSION = "POC-LateDelivery-Regression-AutoML"
MODEL_NAME_CLASSIFICATION = "POC-LateDelivery-Classification-AutoML"

# Workspace
ws = fabric.get_workspace_id()

print(f"📊 Semantic Model: {DATASET}")
print(f"🎯 Target Variable: {TARGET_COLUMN}")
print(f"🤖 Regression Model: {MODEL_NAME_REGRESSION}")
print(f"🤖 Classification Model: {MODEL_NAME_CLASSIFICATION}")
print(f"🏢 Workspace ID: {ws}")

### ⭐ 3. Load Training Data Using DAX

Load **closed deliveries** (with GI Date) for model training.

In [ ]:
# DAX query: Get closed deliveries (deliveries that have shipped)
# Filter for deliveries with Goods Issue (GI) Date populated
dax_query = """
EVALUATE
FILTER(
    Aging,
    NOT(ISBLANK(Aging[GI Date]))
)
"""

# Execute DAX query
print("Loading closed delivery data from semantic model...")
df = fabric.evaluate_dax(dataset=DATASET, dax_string=dax_query, workspace=ws)

# Clean column names (DAX adds table prefixes like 'Aging[column]')
df.columns = [col.split('[')[-1].replace(']', '') if '[' in col else col for col in df.columns]

print(f"✅ Loaded {len(df):,} closed deliveries")
print(f"✅ Columns: {df.shape[1]}")
df.head()

In [ ]:
print("\n=== Dataset Summary ===")
print(f"Total rows: {len(df):,}")
print(f"Columns: {df.shape[1]}")
print(f"\nFirst 20 column names:")
print(df.columns.tolist()[:20])

### ⭐ 4. Data Quality Check & Filtering

Remove rows with missing target values.

In [ ]:
# Check if target column exists
if TARGET_COLUMN not in df.columns:
    print(f"❌ ERROR: Target column '{TARGET_COLUMN}' not found in data!")
    print(f"\nAvailable columns:")
    print(df.columns.tolist())
    raise ValueError(f"Target column '{TARGET_COLUMN}' not found. Update TARGET_COLUMN variable.")

# Check for nulls in target
print(f"\n=== Target Column: {TARGET_COLUMN} ===")
print(f"Total rows: {len(df):,}")
print(f"Null values: {df[TARGET_COLUMN].isnull().sum():,}")
print(f"Non-null values: {df[TARGET_COLUMN].notna().sum():,}")

# Remove rows with null target
df_clean = df[df[TARGET_COLUMN].notna()].copy()
print(f"\n✅ After removing nulls: {len(df_clean):,} rows")

# Basic statistics
print(f"\n=== Target Variable Statistics ===")
print(df_clean[TARGET_COLUMN].describe())

### ⭐ 5. Feature Selection

Define features based on available columns in the Aging table.

In [ ]:
# Define features based on actual Aging table schema
# Updated to match the customer's semantic model columns

potential_features = [
    # Location & Routing
    'Plant',                    # Distribution center
    'Shipping Point',           # Shipping location
    'EWM_CARRIER_CODE',         # Carrier code (important for performance)
    
    # Product Information
    'Brand',                    # Calculated column (Callaway/Odyssey, Jack Wolfskin, etc.)
    'Channel',                  # Calculated column (E-commerce, Inter-company, etc.)
    'Product Category',         # Product category
    'Product Type',             # Product type
    'Standard Or Custom',       # Standard or custom product
    
    # Customer & Account
    'STRATEGIC_ACCOUNT',        # Strategic vs non-strategic (KEY for prioritization!)
    'Sold To - Key',            # Customer identifier
    
    # Delivery Attributes
    'Delivery Type',            # Type of delivery
    'DELIVERY_QTY',             # Quantity (numeric)
    'DELIVERY_VALUE_USD',       # Value in USD (numeric)
    'Delivery Priority',        # Priority level
    'Shipping Condition',       # Shipping condition code
    
    # Processing Status
    'Credit Status',            # Calculated column (Credit checked, Released, etc.)
    'Distribution Status',      # Calculated column (Confirmed, Distributed, etc.)
    'STATUS',                   # Delivery status
]

# Add temporal features from 'Delivery Created On' if it exists
if 'Delivery Created On' in df_clean.columns:
    try:
        df_clean['created_dayofweek'] = pd.to_datetime(df_clean['Delivery Created On']).dt.dayofweek
        df_clean['created_month'] = pd.to_datetime(df_clean['Delivery Created On']).dt.month
        potential_features.extend(['created_dayofweek', 'created_month'])
        print("✅ Added temporal features (day of week, month)")
    except Exception as e:
        print(f"⚠️ Could not create temporal features: {e}")

# Filter to only features that exist in the dataframe
feature_cols = [f for f in potential_features if f in df_clean.columns]

print(f"=== Feature Selection ===")
print(f"Potential features: {len(potential_features)}")
print(f"Available features: {len(feature_cols)}")
print(f"\nUsing features:")
for i, f in enumerate(feature_cols, 1):
    print(f"  {i}. {f}")

# Check for missing features
missing_features = [f for f in potential_features if f not in df_clean.columns]
if missing_features:
    print(f"\n⚠️  Missing features (not in data):")
    for f in missing_features:
        print(f"     - {f}")

### ⭐ 6. Prepare Features + Target

In [ ]:
# Extract features and target
X = df_clean[feature_cols].copy()
y = df_clean[TARGET_COLUMN].copy()

# Handle categorical variables - convert to numeric codes
categorical_cols = X.select_dtypes(include=['object', 'string']).columns.tolist()

print(f"\n=== Categorical Feature Encoding ===")
print(f"Categorical columns: {len(categorical_cols)}")

for col in categorical_cols:
    # Fill NaN with a placeholder before encoding
    X[col] = X[col].fillna('Unknown')
    X[col] = X[col].astype("category").cat.codes
    print(f"  ✓ Encoded: {col}")

# Handle numeric NaNs
numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
for col in numeric_cols:
    if X[col].isnull().sum() > 0:
        median_val = X[col].median()
        X[col] = X[col].fillna(median_val)
        print(f"  ✓ Filled NaNs in {col} with median: {median_val}")

print(f"\n✅ Features: {X.shape[1]} columns, {X.shape[0]:,} rows")
print(f"✅ Target: {y.shape[0]:,} values")
print(f"\nFeature dtypes:")
print(X.dtypes)

### ⭐ 7. Train/Test Split

Split into **training** (80%) and **test** (20%) sets.

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

print(f"✅ Training set: {X_train.shape[0]:,} rows")
print(f"✅ Test set: {X_test.shape[0]:,} rows")
print(f"✅ Features: {X_train.shape[1]} columns")

### ⭐ 8. Train AutoML Model

Using FLAML AutoML to find the best regression model.

**AutoML Settings:**
- **Time budget:** 180 seconds (3 minutes)
- **Metric:** MAE (Mean Absolute Error)
- **Models:** Random Forest, XGBoost, Extra Trees
- **Task:** Regression (predicting numeric aging days)

In [ ]:
# Initialize MLflow experiment
mlflow.set_experiment("AgingPrediction")

# Start MLflow run
with mlflow.start_run(run_name="automl_aging_prediction") as run:
    
    # Configure AutoML
    automl = AutoML()
    
    settings = {
        "time_budget": 180,  # 3 minutes
        "metric": "mae",
        "task": "regression",
        "estimator_list": ["rf", "xgboost", "extra_tree"],
        "log_file_name": "automl_aging.log",
        "seed": 42
    }
    
    # Log parameters
    mlflow.log_params(settings)
    
    print("🚀 Starting AutoML training...")
    print(f"   Time budget: {settings['time_budget']} seconds")
    print(f"   Metric: {settings['metric']}")
    print(f"   Models: {settings['estimator_list']}")
    
    # Train
    automl.fit(X_train, y_train, **settings)
    
    print(f"\n✅ Training complete!")
    print(f"   Best model: {automl.best_estimator}")
    print(f"   Best config: {automl.best_config}")
    
    run_id = run.info.run_id
    print(f"\n📊 MLflow Run ID: {run_id}")

### ⭐ 9. Evaluate Model Performance

Test the model on data it hasn't seen before.

**Metrics:**
- **MAE** (Mean Absolute Error): Average days off in predictions
- **RMSE** (Root Mean Squared Error): Penalizes large errors more
- **R²** (R-squared): How much variance explained (0-1 scale)
- **MAPE** (Mean Absolute Percentage Error): Average % error

In [ ]:
# Make predictions
preds = automl.predict(X_test)

# Calculate metrics
mae = mean_absolute_error(y_test, preds)
rmse = np.sqrt(mean_squared_error(y_test, preds))
r2 = r2_score(y_test, preds)
mape = mean_absolute_percentage_error(y_test, preds) * 100

# Display results
print("\n=== MODEL PERFORMANCE ===")
print(f"MAE:  {mae:.3f} days  ← Average prediction error")
print(f"RMSE: {rmse:.3f} days  ← Larger errors penalized more")
print(f"R²:   {r2:.3f}        ← Variance explained (higher is better)")
print(f"MAPE: {mape:.2f}%      ← Average percentage error")
print("="*40)

### ⭐ 10. Register Model in MLflow

In [ ]:
# Log metrics and register model
with mlflow.start_run(run_id=run_id):
    # Log metrics
    mlflow.log_metric("test_mae", mae)
    mlflow.log_metric("test_rmse", rmse)
    mlflow.log_metric("test_r2", r2)
    mlflow.log_metric("test_mape", mape)
    
    # Log feature names
    mlflow.log_param("features", ",".join(feature_cols))
    mlflow.log_param("num_features", len(feature_cols))
    mlflow.log_param("target_variable", TARGET_COLUMN)
    
    # Register model
    print(f"\n🔄 Registering model: {MODEL_NAME}")
    
    mlflow.sklearn.log_model(
        sk_model=automl.model,
        artifact_path="model",
        registered_model_name=MODEL_NAME
    )
    
    print(f"✅ Model registered successfully!")
    print(f"   Model name: {MODEL_NAME}")
    print(f"   Run ID: {run_id}")

In [ ]:
# Print summary
print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
print(f"Target Variable: {TARGET_COLUMN}")
print(f"Features Used: {len(feature_cols)}")
print(f"Training Samples: {len(X_train):,}")
print(f"Test Samples: {len(X_test):,}")
print(f"\nBest Model: {automl.best_estimator}")
print(f"\nPerformance:")
print(f"  MAE:  {mae:.3f} days")
print(f"  RMSE: {rmse:.3f} days")
print(f"  R²:   {r2:.3f}")
print(f"  MAPE: {mape:.2f}%")
print(f"\nMLflow:")
print(f"  Run ID: {run_id}")
print(f"  Model Name: {MODEL_NAME}")
print("="*60)

### ⭐ 11. Visualizations

#### Actual vs Predicted Scatter Plot

Shows how well predictions match actual values.
- Points on the diagonal line = perfect predictions
- Points above line = model under-predicted (actual > predicted)
- Points below line = model over-predicted (predicted > actual)

In [ ]:
# Scatter plot: Actual vs Predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_test, preds, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel(f'Actual {TARGET_COLUMN} (days)', fontsize=12)
plt.ylabel(f'Predicted {TARGET_COLUMN} (days)', fontsize=12)
plt.title('Actual vs Predicted Aging Days', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"MAE: {mae:.2f} days | RMSE: {rmse:.2f} days | R²: {r2:.3f}")

#### Feature Importance

Shows which features have the most impact on predictions.

In [ ]:
# Feature importance (if available)
if hasattr(automl.model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': automl.model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    plt.barh(importance_df['feature'][:15], importance_df['importance'][:15])
    plt.xlabel('Importance', fontsize=12)
    plt.title('Top 15 Feature Importance', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 Most Important Features:")
    print(importance_df.head(10).to_string(index=False))
else:
    print("Feature importance not available for this model type.")

---

## ✅ Training Complete!

The model has been trained and registered in MLflow.

**Next Steps:**
1. Go to your Fabric workspace → Experiments → AgingPrediction
2. View run details, metrics, and artifacts
3. Proceed to **`03_batch_scoring_pipeline.ipynb`** to generate predictions